In [2]:
from __future__ import print_function
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam

/Library/Python/2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
def tensorflow_test():
    hello = tf.constant('Hello, TensorFlow!')
    sess = tf.Session()
    print(sess.run(hello))

In [25]:
class test:
    def get_data(self):
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        x_train = x_train.reshape(60000, 784)
        x_test = x_test.reshape(10000, 784)
        self.y_train = y_train.reshape(60000, 1)
        self.y_test = y_test.reshape(10000, 1)
        self.x_train = x_train.astype('float32')
        self.x_test = x_test.astype('float32')
        self.x_train /= 255
        self.x_test /= 255
        print(self.x_train[0:2].shape)
        print(self.y_train.shape)
        print(self.x_train[59999])
        print(self.y_train[0:5])

In [26]:
t = test()
t.get_data()

(2, 784)
(60000, 1)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.

In [30]:
from __future__ import print_function
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adam
import numpy as np
import pandas
import matplotlib.pyplot as plt

class distributed_model_training:

	def __init__(self):
		self.num_classes = 10
		self.total_num_epochs = 2
		self.batch_size = 100
		self.get_data()
		self.distribute_data()
		self.define_models()
# 		self.train_model_aggregate()

	def tensorflow_test(self):
		hello = tf.constant('Hello, TensorFlow!')
		sess = tf.Session()
		print(sess.run(hello))

	def get_data(self):
		(x_train, y_train), (x_test, y_test) = mnist.load_data()
		x_train = x_train.reshape(60000, 784)
		x_test = x_test.reshape(10000, 784)
		self.y_train = y_train.reshape(60000, 1)
		self.y_test = y_test.reshape(10000, 1)
		self.x_train = x_train.astype('float32')
		self.x_test = x_test.astype('float32')
		self.x_train /= 255
		self.x_test /= 255
		#One-hot encode the y vectors
		self.y_train = keras.utils.to_categorical(self.y_train, self.num_classes)
		self.y_test = keras.utils.to_categorical(self.y_test, self.num_classes)

	def distribute_data(self):
		#only deal with x_train and y_train
		# np.random.shuffle(self.x_train)
		self.segment_batches = {}
		for i in range(10):
			self.segment_batches["seg"+str(i)] = (self.x_train[6000*i:6000*i+6000], self.y_train[6000*i:6000*i+6000])

	def define_models(self):
		self.segment_models = {}
		for i in range(10):
			model = Sequential()
			model.add(Dense(512, activation='relu', input_shape=(784,)))
			model.add(Dropout(0.2))
			model.add(Dense(512, activation='relu'))
			model.add(Dropout(0.2))
			model.add(Dense(self.num_classes, activation='softmax'))

			model.compile(loss='categorical_crossentropy',
	            optimizer=Adam(),
	            metrics=['accuracy'])

			self.segment_models["seg"+str(i)] = model

	def train_model_aggregate(self):
		for i in range(self.total_num_epochs):
			for segment in self.segment_models:
				# model_seg = self.segment_models[segment]
				(x_train_seg, y_train_seg) = self.segment_batches[segment]				
				# self.segment_models[segment] = self.train_model_segment(model_seg, x_train_seg, y_train_seg)
				history = self.segment_models[segment].fit(x_train_seg, y_train_seg,
			        batch_size=self.batch_size,
			        epochs=1,
			        verbose=0,
			        validation_data=(self.x_test, self.y_test))

		#--------------------------------------------------------------------------------

		for segment in self.segment_models:
			score = self.segment_models[segment].evaluate(self.x_test, self.y_test, verbose=0)
			print('[Segment ' + str(segment) + '] Test loss: ' + str(score[0]))
			print('[Segment ' + str(segment) + '] Test accuracy: ' + str(score[1]))

		#--------------------------------------------------------------------------------
    
		aggregate_model = Sequential()
		aggregate_model.add(Dense(512, activation='relu', input_shape=(784,)))
		aggregate_model.add(Dropout(0.2))
		aggregate_model.add(Dense(512, activation='relu'))
		aggregate_model.add(Dropout(0.2))
		aggregate_model.add(Dense(self.num_classes, activation='softmax'))

		aggregate_model.compile(loss='categorical_crossentropy',
            optimizer=Adam(),
            metrics=['accuracy'])

		print(len(self.segment_models))

		a = [self.segment_models[segment].get_weights() for segment in self.segment_models]
		print(type(a[0]))

		avg_weights = sum([self.segment_models[segment].get_weights() for segment in self.segment_models])/len(self.segment_models)
		print(avg_weights.shape)

		# count = 0
		# for segment in self.segment_models:
		# 	model_seg = self.segment_models[segment]
		# 	seg_weights = model_seg.get_weights()
		# 	print(seg_weights.shape)
		# 	if not weights:
		# 		weights = seg_weights
		# 		count += 1
		# 	else:
		# 		weights = (weights*count + seg_weights)
		# 		count += 1
		# 		weights /= count 


	def train_model_segment(self, model, x_train, y_train):
		return model


model_instance = distributed_model_training()
# model_instance.get_data()

In [33]:
a = [np.array(model_instance.segment_models[segment].get_weights()) for segment in model_instance.segment_models]
print(type(a[0]))

<type 'numpy.ndarray'>


In [34]:
print([type(a[i]) for i in range(10)])

[<type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>, <type 'numpy.ndarray'>]


In [36]:
sum(a)

array([array([[ 0.01635302, -0.0329359 , -0.13793775, ..., -0.16930667,
         0.14292729, -0.09311039],
       [-0.06551155,  0.2207056 ,  0.20803404, ...,  0.09243075,
         0.0580543 ,  0.15351951],
       [ 0.07777649, -0.08474934,  0.20699377, ...,  0.16509902,
        -0.06108699, -0.02703522],
       ...,
       [ 0.18027064,  0.07722271, -0.02912642, ...,  0.01579388,
        -0.14919353,  0.07266685],
       [ 0.0278835 , -0.05178187,  0.22234823, ...,  0.03538033,
         0.10468534,  0.08430828],
       [-0.15660337, -0.00164034,  0.14003426, ...,  0.17141129,
        -0.14533463, -0.04591301]], dtype=float32),
       array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [38]:
print(sorted(model_instance.segment_models))

['seg0', 'seg1', 'seg2', 'seg3', 'seg4', 'seg5', 'seg6', 'seg7', 'seg8', 'seg9']


In [9]:
aggregate_model = Sequential()
aggregate_model.add(Dense(512, activation='relu', input_shape=(784,)))
aggregate_model.add(Dropout(0.2))
aggregate_model.add(Dense(512, activation='relu'))
aggregate_model.add(Dropout(0.2))
aggregate_model.add(Dense(10, activation='softmax'))
print([i.shape for i in aggregate_model.get_weights()])

[(784, 512), (512,), (512, 512), (512,), (512, 10), (10,)]


In [12]:
from matplotlib.mlab import PCA
import numpy as np
data = np.array(np.random.randint(10,size=(10,3)))
results = PCA(data)
print(results)

<matplotlib.mlab.PCA instance at 0x11345de18>


In [ ]:
def PCA(data, dims_rescaled_data=2):
    """
    returns: data transformed in 2 dims/columns + regenerated original data
    pass in: data as 2D NumPy array
    """
    import numpy as NP
    from scipy import linalg as LA
    m, n = data.shape
    # mean center the data
    data -= data.mean(axis=0)
    # calculate the covariance matrix
    R = NP.cov(data, rowvar=False)
    # calculate eigenvectors & eigenvalues of the covariance matrix
    # use 'eigh' rather than 'eig' since R is symmetric, 
    # the performance gain is substantial
    evals, evecs = LA.eigh(R)
    # sort eigenvalue in decreasing order
    idx = NP.argsort(evals)[::-1]
    evecs = evecs[:,idx]
    # sort eigenvectors according to same index
    evals = evals[idx]
    # select the first n eigenvectors (n is desired dimension
    # of rescaled data array, or dims_rescaled_data)
    evecs = evecs[:, :dims_rescaled_data]
    # carry out the transformation on the data using eigenvectors
    # and return the re-scaled data, eigenvalues, and eigenvectors
    return NP.dot(evecs.T, data.T).T, evals, evecs

def test_PCA(data, dims_rescaled_data=2):
    '''
    test by attempting to recover original data array from
    the eigenvectors of its covariance matrix & comparing that
    'recovered' array with the original data
    '''
    _ , _ , eigenvectors = PCA(data, dim_rescaled_data=2)
    data_recovered = NP.dot(eigenvectors, m).T
    data_recovered += data_recovered.mean(axis=0)
    assert NP.allclose(data, data_recovered)


def plot_pca(data):
    from matplotlib import pyplot as MPL
    clr1 =  '#2026B2'
    fig = MPL.figure()
    ax1 = fig.add_subplot(111)
    data_resc, data_orig = PCA(data)
    ax1.plot(data_resc[:, 0], data_resc[:, 1], '.', mfc=clr1, mec=clr1)
    MPL.show()
    
    
